### Enter full names of group members:

##### Name A:
##### Name B:

In [ ]:
import math
import numpy as np
from sympy import prime
from pathlib import Path  # for paths of files
import csv
import copy
import random
from sklearn.metrics.pairwise import cosine_similarity

# ANSI escape codes for colors
class colors:
    red = '\033[91m'
    green = '\033[92m'
    blue = '\033[94m'
    end = '\033[0m'  

### 1. DGIM

#### 1.1. DGIM algorithm

In [ ]:
# Default DGIM parameters

stream_path = 'data/my_stream.txt'

# The window size
N = 500 

In [ ]:
def dgim_algorithm(stream_path, N):
    
    max_buckets = int(N.bit_length())  
    buckets = [[] for _ in range(max_buckets)] 
    current_timestamp = 0

    def add_bucket(timestamp):
        timestamp = timestamp % N  
        i = 0
        carry = [timestamp]  
        while carry:
            if i >= len(buckets):
                buckets.append([]) 
            if len(buckets[i]) < 2:
                buckets[i].extend(carry)  
                break
            else:
                min_time = min(buckets[i])
                buckets[i] = [t for t in buckets[i] if t != min_time]  
                carry = [min_time]  
            i += 1
        remove_exceeding_buckets()

    def remove_exceeding_buckets():
        for i in range(len(buckets)):
            buckets[i] = [t for t in buckets[i] if (current_timestamp - t) % N < N]

    with open(stream_path, 'r') as f:
        bit = f.read(1)
        while bit:
            current_timestamp += 1
            if bit == '1':
                add_bucket(current_timestamp)
            bit = f.read(1)

    end_timestamp = current_timestamp % N
    bucket_list = [sorted(bucket, reverse=True) for bucket in buckets if bucket]
    
    return bucket_list, end_timestamp

In [ ]:
bucket = dgim_algorithm(stream_path, N)

In [ ]:
print(f"The updated list of timestamps buckets from DGIM algorithm: \n {bucket[0]}")
print(f"The end timestamp: {bucket[1]}")   

#### 1.2. Query the Bucket 

In [ ]:
def actual_count(stream_path, k):
    stream_list = []
    with open(stream_path, 'r') as file:
        for line in file:
            stream_list.extend(list(map(int, line.strip())))

    # Convert the list into a numpy array
    stream_array = np.array(stream_list)
    
    return int(np.sum(stream_array[-k:]))

In [ ]:
def dgim_query(bucket, N, k):  
    
    # Extract the buckets and the end timestamp
    bucket_list, end_time_stamp = bucket
   
    
    # To-do! initialize the different variables
    

    # To-do! query the dgim bucket using the k parameters
    
    
    return math.ceil(one_count)

In [ ]:
# List of queries
K = [10, 50, 100, 300, 500] 

In [ ]:
print("---------------------------------------------------------------")
for k in K:
    dgim_count = dgim_query(bucket, 500, k)
    true_count = actual_count(stream_path, k)
    
    print(f"The total 1s in the last {k} bits by DGIM: {dgim_count}")
    print(f"The true count of 1s in the last {k} bits: {true_count}")
    print(f"The DGIM error for predicted 1s in the last {k} bits: \
    {round(abs(100*(dgim_count-true_count))/true_count,2)} %")
    print("---------------------------------------------------------------")

### 2. Bloom filters

In [ ]:
# Username data for the creation of bloom filters - B
data_file = (Path("data/bloom_username").with_suffix('.csv'))

# Test data to check the functionality and false positive rate
test1_file = (Path("data/test1_username").with_suffix('.csv'))
test2_file = (Path("data/test2_username").with_suffix('.csv'))

# Default bloom filter parameters
bloom_size = 1500000 # parameter N
h = 3 # number of hash functions

In [ ]:
# create an array of bloom filter with zeros
B = np.zeros(bloom_size)

In [ ]:
B

#### 2.1. Create Bloom filter

In [ ]:
def generate_hash(h, N):
    hash_list = []
    primes = [114649, 914843, 1382753]
    for j in range(h):
        hash_list.append(lambda s, j=j: sum(ord(s[i]) * primes[j]**i for i in range(len(s))) % N)
    return hash_list

In [ ]:
hashes = generate_hash(h, bloom_size)

In [ ]:
def create_bloom_filter(B, hashes, data):
    with data.open() as f:
        for name in f:
            for hash in hashes:
                B[hash(name)] = 1
    return B

In [ ]:
bloom_array = create_bloom_filter(B, hashes, data_file)

In [ ]:
bloom_array

#### 2.2. Verify usernames

In [ ]:
def single_verify_username(bloom_array, hashes, new_user):
    
    # To-do! verify username and return a code of 0 or 1 (1 - username taken and 0 - username available)
    for hash in hashes:
        if (bloom_array[hash(new_user)] == 0):
            return 0
    return 1
    

In [ ]:
# Feel free to test different usernames here

# new_username = "KazeemTDT4305"

new_username = "jlrsjrvljvrewrw"

In [ ]:
user_code = single_verify_username(bloom_array, hashes, new_username)

In [ ]:
if user_code == 1:
    print(colors.red + f"Username {new_username} has been taken. Try again!" + colors.end)
elif user_code == 0:
    print(colors.green + f"Username {new_username} is available. Congrats!" + colors.end)
else:
    print(colors.blue + f"Wrong pass code. Please reverify!" + colors.end)  

In [ ]:
def group_verify_username(bloom_array, hashes, data):
    # Initialize counts
    total_name = 0
    taken_name = 0
    
    with data.open() as f:
        for name in f:
            hash_hit_count = 0
            for hash in hashes:
                if (bloom_array[hash(name)] == 0):
                    break
                elif (bloom_array[hash(name)] == 1):
                    hash_hit_count += 1
            total_name += 1
            if (hash_hit_count == 3):
                taken_name += 1
            
    return round(taken_name/total_name*100,2)   

In [ ]:
print("----------------------------------------------------------")
user_total = group_verify_username(bloom_array, hashes, test1_file)
print(f"Percentage of username seen before from test 1: {user_total}%")
print("----------------------------------------------------------")
user_total = group_verify_username(bloom_array, hashes, test2_file)
print(f"Percentage of username seen before from test 2: {user_total}%")
print("----------------------------------------------------------")

### 3. Flajolet-Martin

In [ ]:
def flajolet_martin(input_stream):
    R = 0  # Initialize maximum rightmost zero bit position to 0

    # To-do! Define hash function h(x) = 6x + 1 mod 5
    hash_func = lambda x: (6 * x + 1) % 5

    # To-do! Iterate over the input stream and update maximum rightmost zero bit position
    for element in input_stream:
        hash_value = hash_func(element)
        binary_hash = bin(hash_value)[2:]
        rightmost_zero_position = binary_hash.rfind('0')

        if rightmost_zero_position > R:
            R = rightmost_zero_position

    # Estimate the number of distinct elements
    distinct_estimate = 2 ** R

    return distinct_estimate

In [ ]:
# Input stream
input_stream1 = [1, 1, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1]
input_stream2 = [1, 3, 2, 1, 2, 3, 4, 3, 1, 2, 3, 1]

# Run the Flajolet-Martin algorithm
distinct_estimate1 = flajolet_martin(input_stream1)
distinct_estimate2 = flajolet_martin(input_stream2)

# Print the estimated number of distinct elements
print("-----------------------------------------------------")
print(f"Distinct elements (estimated) in input stream 1:", distinct_estimate1)
print("-----------------------------------------------------")
print(f"Distinct elements (estimated) in input stream 2:", distinct_estimate2)
print("-----------------------------------------------------")

### 4. Adword 

#### 4.1. Greedy Algorithm

In [ ]:
# User queries
queries = ["big data", "big data", "big data","bloom filters", "bloom filters", "bloom filters",
           "flajolet martin", "flajolet martin", "flajolet martin", "dgim algorithm", "dgim algorithm", "dgim algorithm"]

In [ ]:
# Company A B C and D keywords and budget $$$
global_companies = {
        'A': ["big data", "bloom filters", 3],
        'B': ["flajolet martin", 3],
        'C': ["flajolet martin", "dgim algorithm", 3],
        'D': ["big data", 3],
    }

In [ ]:
def greedy_algorithm(local_companies, queries):
    # Initial revenue
    revenue = 0
    
    # To-do! update revenue using greedy algorithm
    
    return revenue

In [ ]:
total_revenue = 0
total_trials = 10
print("Starting trials using Greedy Algorithm...")
print("------------------------------------------------")
for i in range(total_trials):
    local_companies = copy.deepcopy(global_companies)
    revenue = greedy_algorithm(local_companies, queries)
    total_revenue = total_revenue + revenue
    print(f"Trial {i+1} - Revenue generated: {revenue}")
print("------------------------------------------------")   
print("Average revenue generated for all trials: ",total_revenue/total_trials)

#### 4.2. Balance Algorithm

In [ ]:
def balance_algorithm(local_companies, queries):
    # Initial revenue
    revenue = 0
    
    # To-do! update revenue using balance algorithm
    cosine_similarity
    
    return revenue

In [ ]:
total_revenue = 0
total_trials = 10
print("Starting trials using Balance Algorithm...")
print("-------------------------------------------")
for i in range(total_trials):
    local_companies = copy.deepcopy(global_companies)
    revenue = balance_algorithm(local_companies, queries)
    total_revenue = total_revenue + revenue
    print(f"Trial {i+1} - Revenue generated: {revenue}")
print("-------------------------------------------")   
print("Average revenue generated for all trials: ",total_revenue/total_trials)

### 5. Recommender System

In [ ]:
# Ratings matrix (each row corresponds to a movie, and each column corresponds to a user)
ratings_matrix = np.array([
    [1, 0, 3, 0, 0, 5, 0, 0, 5, 0, 4, 0],
    [0, 0, 5, 4, 0, 0, 4, 0, 0, 2, 1, 3],
    [2, 4, 0, 1, 2, 0, 3, 0, 4, 3, 5, 0],
    [0, 2, 4, 0, 5, 0, 0, 4, 0, 0, 2, 0],
    [0, 0, 4, 3, 4, 2, 0, 0, 0, 0, 2, 5],
    [1, 0, 3, 0, 3, 0, 0, 2, 0, 0, 4, 0]
])

#### 5.1. User-User Collaborative Filtering

In [ ]:
def user_cf(rate_m, tup_mu, neigh):
    
    # To-do! implement a user-user CF using cosine similarity as distance measure
    user_index, movie_index = tup_mu
    num_users, num_movies = rate_m.shape

    normalized_ratings = (rate_m - np.mean(rate_m, axis=1, keepdims=True)) / np.std(rate_m, axis=1, keepdims=True)
    cosine_sim = np.dot(normalized_ratings, normalized_ratings.T)
    norms = np.sqrt(np.diagonal(cosine_sim))
    print(norms)
    cosine_sim /= np.outer(norms, norms)
    np.fill_diagonal(cosine_sim, 1) 
    
    similar_indices = np.argsort(-cosine_sim[user_index])[:neigh+1]
    if user_index in similar_indices:
        similar_indices = similar_indices[similar_indices != user_index]
    else:
        similar_indices = similar_indices[:-1]

    # Step 3: Predict the rating using weighted average of ratings by similar users
    similar_users_similarities = cosine_sim[user_index, similar_indices]
    similar_users_ratings = rate_m[similar_indices, movie_index]
    
    # Avoid division by zero by adding a small number to the denominator
    prediction = np.dot(similar_users_similarities, similar_users_ratings) / (np.sum(similar_users_similarities) + 1e-10)
    
    
    return prediction   

In [ ]:
# List of tuple of movie rating by users to be predicted e.g (1, 5) refers to the rating of movie 1 by user 5
list_mu_query = [(1, 5), (3, 3)]

# Neighbor selection (|N|)
neigh = 2

In [ ]:
print("-----------------------------------------------------------------")   
for mu_query in list_mu_query:
    predicted_rating = user_cf(ratings_matrix, mu_query, neigh)
    print(f"The predicted rating of movie {mu_query[0]} by user {mu_query[1]}: {predicted_rating} (User-User CF)")
    print("-----------------------------------------------------------------")   

#### 5.2. Item-Item Collaborative Filtering

In [ ]:
def item_cf(rate_m, tup_mu, neigh):
    
    # To-do! implement a item-item CF using cosine similarity as distance measure
    item_index, user_index = tup_mu  # This time the first entry is the item and the second is the user
    num_items, num_users = rate_m.shape


    normalized = (rate_m.T - np.mean(rate_m.T, axis=1, keepdims=True)) / np.std(rate_m.T, axis=1, keepdims=True)
    sim = np.dot(normalized, normalized.T)
    norms = np.sqrt(np.diagonal(sim))
    sim /= np.outer(norms, norms)

    similar_indices = np.argsort(-sim[item_index])[:neigh+1]
    if item_index in similar_indices:
        similar_indices = similar_indices[similar_indices != item_index]
    else:
        similar_indices = similar_indices[:-1]

    similar_items_similarities = sim[item_index, similar_indices]
    similar_items_ratings = rate_m[similar_indices, user_index]

    if np.sum(similar_items_similarities) == 0:
        prediction = np.mean(rate_m[item_index])  # Fallback to average rating if no similarities
    else:
        prediction = np.dot(similar_items_similarities, similar_items_ratings) / (np.sum(similar_items_similarities) + 1e-10)

    return prediction

    
    return prediction

In [ ]:
print("-----------------------------------------------------------------")   
for mu_query in list_mu_query:
    predicted_rating = item_cf(ratings_matrix, mu_query, neigh)
    print(f"The predicted rating of movie {mu_query[0]} by user {mu_query[1]}: {predicted_rating} (Item-Item CF)")
    print("-----------------------------------------------------------------")   

### Provide concise answers to all 5 cases in the Project 3 description below

#### Case 1

In [ ]:
# Enter answer here

#### Case 2

In [ ]:
# Enter answer here

#### Case 3

In [ ]:
# Enter answer here

#### Case 4

In [ ]:
# Enter answer here

#### Case 5

In [ ]:
# Enter answer here